  
# Water Depth Estimation using Remote Sensing and DEM
## Simplistic method
 
**Author:** Jin Teng (jin.teng@csiro.au)

**Compatability:** Notebook currently compatible with the NCI.

**Dependencies:** The code below requires installation of following packages:

    rasterio
    numpy
    scipy


## Description

This is a simplistic tool that estimate water depth from satellite imagery and DEM. It is developed to add value to remote sensing (RS) analysis by calculating water depth based solely on an inundation map with an associated digital elevation model (DEM).

The main steps of the algorithm are:

    (1) extract the polygons for the inundation extent from the raster; 
    (2) extract the DEM value (elevation) for these polygons;
    (3) calculate the maximum elevation for each and all polygons;
    (4) calculate floodwater depth by subtracting topographic elevation from the maximum elevation.

We have included a run-down of the tool that is demonstrated in the notebook. 

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

## Load packages

Import Python packages that are used for the analysis.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import time
import rasterio
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from scipy import ndimage

## Set input variables

In [ ]:
FloodExtent_tif = os.path.join('data', 'FloodExtent_modified.tif')
DEM_tif = os.path.join('data', 'Elevation.tif')
output_path = os.path.join('data', 'output_wd_simplistic_all.tif')
WOfS_nodata_value = 0
WOfS_dry_value = 2
WOfS_wet_value = 3


In [ ]:
print(os. getcwd())

## Read flood extent raster

In [ ]:
start_time = time.time()
with rasterio.open(FloodExtent_tif) as src:
    flood_extent = src.read(1)
out_mask = flood_extent!=WOfS_wet_value
wet = np.where(flood_extent==WOfS_wet_value,1,0)
print("--- %s seconds ---" % (time.time() - start_time))
plt.figure(figsize=(10,8))
plt.imshow(flood_extent)
plt.colorbar(label='Water occurance')
plt.title('Flood Extent map')

## Group the flood extent into polygons

In [ ]:
structure = np.ones((3, 3))
groups, num_ids = ndimage.label(wet, structure=structure)
group_ids = np.arange(0, num_ids + 1)
plt.figure(figsize=(10,8))
plt.imshow(groups,cmap='jet')
plt.colorbar()

## Calculating water depth using all and individual polygon

In [ ]:
with rasterio.open(DEM_tif) as src_dem:
    dem = src_dem.read(1, masked = True)

# All
max = np.max(wet * dem)
water_depth_a = max - dem
water_depth_a[water_depth_a<0] = 0
water_depth_a =np.ma.masked_where(out_mask, water_depth_a)
# Individual
groups_max = ndimage.maximum(dem, groups, group_ids) 
water_depth_i = np.array([groups_max[x] for x in groups]) - dem
water_depth_i[water_depth_i<0] = 0
water_depth_i =np.ma.masked_where(out_mask, water_depth_i)
plt.figure(figsize=(10,6))
plt.subplot(121)
plt.imshow(water_depth_a, cmap='terrain')
plt.colorbar(label='Water depth')
plt.title('All')
plt.subplot(122)
plt.imshow(water_depth_i,cmap='terrain')
plt.title('Individual')
plt.colorbar(label='Water depth')
#plt.legend((fig[0]), ('a'),
#           fontsize=12, ncol=1, framealpha=0, fancybox=True)


## Write outputs

In [ ]:
waterdepth_path = os.path.join(os.path.dirname(output_path), 'output_wd_simplistic_all.tif')
with rasterio.open(
    waterdepth_path, 'w',
    driver='GTiff',
    dtype=water_depth_a.dtype,
    count=1,
    nodata=np.nan,
    transform = src_dem.transform,
    width=src_dem.width,
    height=src_dem.height) as dst:
    dst.write(water_depth_a, indexes=1)
waterdepth_path = os.path.join(os.path.dirname(output_path), 'output_wd_simplistic_individual.tif')
with rasterio.open(
    waterdepth_path, 'w',
    driver='GTiff',
    dtype=water_depth_i.dtype,
    count=1,
    nodata=np.nan,
    transform = src_dem.transform,
    width=src_dem.width,
    height=src_dem.height) as dst:
    dst.write(water_depth_i, indexes=1)